In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [6]:
root_agent = Agent(
    name="pet_care_advisor",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that provides pet care advise.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


In [7]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [8]:
response = await runner.run_debug(
    "Tell me how to care for Labrodaore retriever when he is having cramps and be specific in your instructions."
)


 ### Created new session: debug_session_id

User > Tell me how to care for Labrodaore retriever when he is having cramps and be specific in your instructions.
pet_care_advisor > It sounds like you're concerned about your Labrador Retriever experiencing cramps. It's important to address this promptly and safely. Here's how to care for your dog when they are having cramps, with specific instructions:

**Immediate Actions During a Cramp:**

1.  **Stop Activity Immediately:** If your dog is exercising when a cramp occurs, stop all activity. Do not encourage them to continue.
2.  **Gentle Massage:** Once your dog is still and you can safely approach them, gently massage the affected muscle. Use firm but gentle pressure to help the muscle relax. You can use your hands to stroke or knead the area.
3.  **Passive Stretching (with caution):** If your dog is tolerant and the cramp seems to be easing, you can try to very gently help them stretch the affected limb. This should only be done if your

******************THIS is the Starting code for Care Advisor Agent and Instructor Subagent ***********************


In [9]:
import os
import json
from google import genai
from google.genai import types

# --- 1. Subagent Simulation: Instruction Provider (The Tool) ---

# Mock data simulating the "core care instructions generated earlier"
CORE_INSTRUCTIONS_DB = {
    "dog": {
        "emergency": "If your dog is choking or bleeding severely, muzzle the dog first if possible, and transport immediately to the nearest 24hr vet clinic. Keep the wound clean and apply firm pressure.",
        "diet": "Ensure your dog has fresh water access at all times. Feeding guidelines vary by weight; typically 2-3 cups of high-quality kibble divided into two meals. Avoid chocolate, grapes, and onions.",
        "training": "Use positive reinforcement methods. Keep sessions short (5-10 minutes). Focus on consistency for commands like 'sit', 'stay', and 'come'."
    },
    "cat": {
        "emergency": "For sudden lethargy or inability to urinate, seek immediate veterinary care, as this could be a blocked bladder (especially in male cats). Do not attempt home remedies.",
        "diet": "Cats require taurine, found in meat. Ensure commercial food is labeled complete and balanced. Always keep litter box clean and separate from food/water area.",
        "behavior": "Provide scratching posts (vertical and horizontal) and vertical climbing space. Introduce new pets slowly over a period of 1-2 weeks."
    }
}

def get_core_care_instructions(pet_type: str, situation_keywords: str) -> str:
    """
    Retrieves core care instructions for a specific pet type and situation.
    
    This function simulates the 'Instruction Provider' subagent.
    It accesses a specialized knowledge base to supply relevant parts
    of pre-generated core care instructions to the parent agent.

    Args:
        pet_type: The type of pet (e.g., 'dog', 'cat').
        situation_keywords: Keywords describing the current situation (e.g., 'emergency', 'diet', 'behavior').

    Returns:
        A string containing the relevant core instructions, or a message if none are found.
    """
    pet_type = pet_type.lower()
    
    if pet_type in CORE_INSTRUCTIONS_DB:
        # Check if any keyword matches the known instruction types
        for key, instruction in CORE_INSTRUCTIONS_DB[pet_type].items():
            if key in situation_keywords.lower():
                return f"Core Instruction for {pet_type.capitalize()} ({key}): {instruction}"
        
        # If no specific keyword match, return general advice for the pet
        return f"General Core Instruction for {pet_type.capitalize()}: {CORE_INSTRUCTIONS_DB[pet_type].get('diet', 'No specific core instruction found, but general care is always important.')}"
    
    return f"Pet type '{pet_type}' not found in the core knowledge base."


# --- 2. Parent Agent Simulation: Care Advisor ---

def care_advisor_agent(user_prompt: str):
    """
    The Care Advisor agent (Parent Agent). 
    
    It uses the Gemini model and is guided by a system instruction to provide 
    accurate, high-level guidance. It decides whether to use the 
    'Instruction Provider' subagent (Tool) to gather specific core knowledge.
    """
    print(f"--- Care Advisor Agent Activated ---")
    print(f"User Query: {user_prompt}\n")

    # Initialize the client
    # The API key is typically loaded from the environment variable GEMINI_API_KEY
    try:
        client = genai.Client()
    except Exception as e:
        print("Error initializing Gemini Client. Ensure the GEMINI_API_KEY environment variable is set.")
        print(f"Details: {e}")
        return

    # System instruction for the Care Advisor agent
    system_instruction = (
        "You are the Care Advisor, a compassionate and highly accurate parent agent "
        "for pet care. Your primary goal is to provide specific, accurate, and "
        "actionable guidance based on the user's described situation. "
        "ALWAYS use the provided tools (the Instruction Provider subagent) "
        "if the user's query relates to core care or standard emergency protocols "
        "for dogs or cats to integrate that foundational knowledge into your response."
    )

    # 1. Initial request to the Gemini model
    config = types.GenerateContentConfig(
        system_instruction=system_instruction,
        tools=[get_core_care_instructions] # Register the subagent function as a tool
    )
    
    response = client.models.generate_content(
        model='gemini-2.5-flash', # Or a similar model
        contents=[user_prompt],
        config=config,
    )

    # --- Tool/Subagent Execution Loop (Handles the delegation) ---
    
    while response.function_calls:
        function_call = response.function_calls[0]
        func_name = function_call.name
        func_args = dict(function_call.args)
        
        print(f"🤖 Care Advisor detected need for Subagent...")
        print(f"    -> Calling Instruction Provider (Tool): {func_name}")
        print(f"    -> Arguments: {func_args}\n")

        # Execute the function that simulates the subagent
        if func_name == "get_core_care_instructions":
            tool_output = get_core_care_instructions(**func_args)
        else:
            tool_output = f"Error: Unknown function requested: {func_name}"

        print(f"✅ Instruction Provider (Subagent) returned:\n{tool_output}\n")
        
        # 2. Send the tool's output back to the model for final answer generation
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=[
                user_prompt,
                types.Content(
                    role="model", 
                    parts=[types.Part.from_function_call(function_call)]
                ),
                types.Content(
                    role="tool", 
                    parts=[types.Part.from_function_response(
                        name=func_name, 
                        response={"result": tool_output}
                    )]
                )
            ],
            config=config,
        )

    # --- Final Output ---
    print("--- Final Care Advisor Guidance ---")
    print(response.text)
    print("-----------------------------------")


if __name__ == "__main__":
    # Ensure you have the API key set in your environment:
    # export GEMINI_API_KEY="YOUR_API_KEY"

    # Example 1: Query that requires core instruction retrieval (Dog Diet)
    care_advisor_agent("My puppy is 10 weeks old. What are the general feeding guidelines and are there any human foods I must never give my dog?")
    
    print("\n" + "="*80 + "\n")

    # Example 2: Query that requires core instruction retrieval (Cat Emergency)
    care_advisor_agent("I think my male cat hasn't peed all day and seems very lethargic. What should I do right now?")

    print("\n" + "="*80 + "\n")
    
    # Example 3: Query that does not necessarily require the tool (General behavior advice)
    care_advisor_agent("How do I help my dog stop barking at the mailman? Should I use punishment?")

    print("\nNote: This is a simulation. To run this, you need the google-genai library installed ('pip install google-genai') and your GEMINI_API_KEY environment variable set.")

--- Care Advisor Agent Activated ---
User Query: My puppy is 10 weeks old. What are the general feeding guidelines and are there any human foods I must never give my dog?

--- Final Care Advisor Guidance ---
For your 10-week-old puppy, ensure they have access to fresh water at all times. General feeding guidelines suggest around 2-3 cups of high-quality kibble per day, divided into two meals.

Regarding human foods, you must **never** give your dog chocolate, grapes, or onions.
-----------------------------------


--- Care Advisor Agent Activated ---
User Query: I think my male cat hasn't peed all day and seems very lethargic. What should I do right now?



--- Final Care Advisor Guidance ---
This is a serious emergency, especially for a male cat. The symptoms you're describing (not peeing all day, lethargy) could indicate a blocked bladder, which is life-threatening. You need to seek immediate veterinary care. Do not attempt any home remedies. Please go to the nearest emergency vet clinic right away.
-----------------------------------


--- Care Advisor Agent Activated ---
User Query: How do I help my dog stop barking at the mailman? Should I use punishment?



--- Final Care Advisor Guidance ---
It's understandable that you're looking for solutions to help your dog with barking at the mailman. Regarding your question about punishment, based on core care instructions for dog training, it's best to **avoid punishment**. Instead, focus on using **positive reinforcement methods**. Punishment can often increase anxiety and fear, potentially making the barking worse or leading to other behavioral issues.

Here's a positive reinforcement approach to help manage your dog's barking at the mailman:

1.  **Identify the Trigger and Manage the Environment:**
    *   Observe when your dog starts barking. Is it the sight, sound, or arrival of the mailman?
    *   Consider managing the environment during mail delivery time. This could involve closing curtains or blinds, playing music to mask outdoor sounds, or putting your dog in a different room with a distracting toy (like a Kong filled with peanut butter) before the mailman arrives.

2.  **Desensitize an